In [26]:
import os
import numpy as np
from nibabel.testing import data_path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.transform import resize
import tflearn
import numpy as np
import nibabel as nib
from scipy import ndimage
import os
import zipfile
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random
from scipy import ndimage

In [27]:
# Helper functions


def read_nifti_file(filepath):
    scan = nib.load(filepath)
    scan = scan.get_fdata()
    return scan

def normalize(volume):
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume


def resize_volume(img):
    # Set the desired depth
    desired_depth = 160
    desired_width = 256
    desired_height = 256
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    img = ndimage.rotate(img, 90, reshape=False)
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img


def process_scan(path):
    volume = read_nifti_file(path)
    volume = normalize(volume)
    volume = resize_volume(volume)
    return volume

In [28]:
normal_scan_paths = [
    os.path.join(os.getcwd(), "Data/CN", x)
    for x in os.listdir("Data/CN")
]


abnormal_scan_paths = [
    os.path.join(os.getcwd(), "Data/AD", x)
    for x in os.listdir("Data/AD")
]

print("CN files: " + str(len(normal_scan_paths)))
print("AD files: " + str(len(abnormal_scan_paths)))

CN files: 22
AD files: 18


In [29]:
# Each scan is resized across height, width, and depth and rescaled.
abnormal_scans = np.array([process_scan(path) for path in abnormal_scan_paths])
normal_scans = np.array([process_scan(path) for path in normal_scan_paths])

# assign 1, for the normal ones assign 0.
abnormal_labels = np.array([1 for _ in range(len(abnormal_scans))])
normal_labels = np.array([0 for _ in range(len(normal_scans))])

# Split data 
x_train = np.concatenate((abnormal_scans[:9], normal_scans[:9]), axis=0)
y_train = np.concatenate((abnormal_labels[:9], normal_labels[:9]), axis=0)
x_val = np.concatenate((abnormal_scans[9:], normal_scans[9:]), axis=0)
y_val = np.concatenate((abnormal_labels[9:], normal_labels[9:]), axis=0)
print(
    "Number of samples in train and validation are %d and %d."
    % (x_train.shape[0], x_val.shape[0])
)

Number of samples in train and validation are 18 and 22.


In [30]:
@tf.function
def rotate(volume):
    def scipy_rotate(volume):
        angles = [-20, -10, -5, 5, 10, 20]
        angle = random.choice(angles)
        volume = ndimage.rotate(volume, angle, reshape=False)
        volume[volume < 0] = 0
        volume[volume > 1] = 1
        return volume

    augmented_volume = tf.numpy_function(scipy_rotate, [volume], tf.float32)
    return augmented_volume


def train_preprocessing(volume, label):
    volume = rotate(volume)
    volume = tf.expand_dims(volume, axis=3)
    return volume, label


def validation_preprocessing(volume, label):
    volume = tf.expand_dims(volume, axis=3)
    return volume, label

In [31]:
x_train.shape

(18, 256, 256, 160)

In [32]:
train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
validation_loader = tf.data.Dataset.from_tensor_slices((x_val, y_val))
def load_image(file, label):
    nifti = np.asarray(nibabel.load(file.numpy().decode('utf-8')).get_fdata()).astype(np.float32)

    xs, ys, zs = np.where(nifti != 0)
    nifti = nifti[min(xs):max(xs) + 1, min(ys):max(ys) + 1, min(zs):max(zs) + 1]
    nifti = nifti[0:100, 0:100, 0:100]
    nifti = np.reshape(nifti, (256, 256, 160, 1))
    return nifti, label


@tf.autograph.experimental.do_not_convert
def load_image_wrapper(file, label):
    result_tensors = tf.py_function(load_image, [file, label], [tf.float64, tf.float64])
    result_tensors[0].set_shape([256, 256, 160, 1])
    result_tensors[1].set_shape([None])
    return result_tensors



train_loader = train_loader.map(load_image_wrapper, num_parallel_calls=24)
validation_loader = validation_loader.map(load_image_wrapper, num_parallel_calls=24)

batch_size = 2
# Augment the on the fly during training.
train_dataset = (
    train_loader.shuffle(len(x_train))
    .map(train_preprocessing)
    .batch(batch_size)
)
# Only rescale.
validation_dataset = (
    validation_loader.shuffle(len(x_val))
    .map(validation_preprocessing)
    .batch(batch_size)
)
train_dataset = train_dataset.map(load_image_wrapper, num_parallel_calls=24)
validation_dataset = validation_dataset.map(load_image_wrapper, num_parallel_calls=24)

In [33]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv3D(16, (3,3,3), activation='relu', input_shape=(256, 256, 160, 1)),
    tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
    # The second convolution
    tf.keras.layers.Conv3D(32, (3,3,3), activation='relu'),
    tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
    # The third convolution
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    #tf.keras.layers.Dense(12, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_6 (Conv3D)            (None, 254, 254, 158, 16) 448       
_________________________________________________________________
max_pooling3d_6 (MaxPooling3 (None, 127, 127, 79, 16)  0         
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 125, 125, 77, 32)  13856     
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 62, 62, 38, 32)    0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4674304)           0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 4674305   
Total params: 4,688,609
Trainable params: 4,688,609
Non-trainable params: 0
____________________________________________

In [34]:
model.fit(train_dataset,validation_data=validation_dataset)

MemoryError: bad allocation